In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

spam.csv



In [2]:
df = pd.read_csv("../input/spam.csv",encoding='latin-1')

In [3]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
dict = {'ham':0,'spam':1}
df['v1'] = df['v1'].map(dict)
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
del df['Unnamed: 2']
del df['Unnamed: 3']
del df['Unnamed: 4']

In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
c_vec = CountVectorizer(lowercase=1,min_df=.00001,stop_words='english')
c_vec.fit(df['v2'].values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=1, max_df=1.0, max_features=None, min_df=1e-05,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [7]:
train_df = df[0:5000]
test_df = df[5000:]
test_df.index=(range(test_df.shape[0]))
Y_train = train_df['v1'].values

In [8]:
def prob_y(Y_train,num_class=2):
    p_y = np.zeros([num_class,])
    n_y = np.zeros([num_class,])
    d_y = Y_train.shape[0]
    for i in range(Y_train.shape[0]):
        n_y[Y_train[i]] = n_y[Y_train[i]]+1
    p_y = n_y/d_y
    return p_y

In [9]:
p_y = prob_y(Y_train)
p_y

array([ 0.8654,  0.1346])

In [10]:
def prob_xy(c_vec,train_df,Y_train,num_class=2):
    d_y = np.zeros([num_class,])+len(c_vec.vocabulary_)
    p_xy = np.zeros([num_class,len(c_vec.vocabulary_)])
    for i in np.unique(Y_train):
        temp_df = train_df[train_df['v1']==i]
        temp_x = c_vec.transform(temp_df['v2'].values)
        n_xy = np.sum(temp_x,axis=0)+1
        d_y[i] = d_y[i]+np.sum(temp_x)
        p_xy[i] = n_xy/d_y[i] 
    return p_xy

In [11]:
p_xy = prob_xy(c_vec,train_df,Y_train,2)
p_xy

array([[  2.57944697e-05,   2.57944697e-05,   5.15889393e-05, ...,
          2.57944697e-05,   2.57944697e-04,   5.15889393e-05],
       [  5.77064316e-04,   1.52135138e-03,   5.24603924e-05, ...,
          1.04920785e-04,   5.24603924e-05,   5.24603924e-05]])

In [12]:
def classify(c_vec,test_df,p_xy,p_y,num_class=2):
    pred = []
    pre_yx = []
    for doc in test_df['v2'].values:
        temp_doc = (c_vec.transform([doc])).todense()
        temp_prob = np.zeros([num_class,])
        for i in range(num_class):
            temp_prob[i] = np.prod(np.power(p_xy[i],temp_doc))*p_y[i]
        pred.append(np.argmax(temp_prob))
    return pred

In [13]:
pred = classify(c_vec,test_df,p_xy,p_y,num_class=2)

In [14]:
def accuracy(pred,Y):
    return np.sum(pred==Y)/Y.shape[0]

In [15]:
Y_test = test_df['v1'].values
test_accuracy = accuracy(pred,Y_test)
print('Test Data Accuaracy = '+str(test_accuracy)) 

Test Data Accuaracy = 0.984265734266


In [16]:
pred_train = classify(c_vec,train_df,p_xy,p_y,num_class=2)
print('Train Data Accuracy = '+str(accuracy(pred_train,Y_train)))

Train Data Accuracy = 0.995
